## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = keras.applications.Xception(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()

Using TensorFlow backend.


83689472/83683744 [==============================] - 4s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________

## 添加層數

In [2]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(p=0.1)(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 136


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


## 鎖定特定幾層不要更新權重

In [0]:
for layer in model.layers[:60]:
    layer.trainable = False
for layer in model.layers[60:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)  # (50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    
    return X_train, X_test
    
    
# Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

# OneHot Label 由(None, 1)-(None, 10)
# ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
50000/50000 [==============================] - 71s 1ms/step - loss: 1.4449 - accuracy: 0.4830
Epoch 2/10
50000/50000 [==============================] - 61s 1ms/step - loss: 0.9822 - accuracy: 0.6681
Epoch 3/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.8413 - accuracy: 0.7141
Epoch 4/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.7481 - accuracy: 0.7438
Epoch 5/10
50000/50000 [==============================] - 62s 1ms/step - loss: 0.6568 - accuracy: 0.7749
Epoch 6/10
50000/50000 [==============================] - 60s 1ms/step - loss: 0.5809 - accuracy: 0.7998
Epoch 7/10
50000/50000 [==============================] - 60s 1ms/step - loss: 0.5110 - accuracy: 0.8268
Epoch 8/10
50000/50000 [==============================] - 60s 1ms/step - loss: 0.4453 - accuracy: 0.8475
Epoch 9/10
50000/50000 [==============================] - 60s 1ms/step - loss: 0.3790 - accuracy: 0.8704
Epoch 10/10
50000/50000 [==============================